In [1]:
from HSI_utils import *
from losses import *
from timeit import default_timer as timer
import conv_models_AD_full_HSI
import VAE_model_trainer_full_HSI

from datetime import datetime, timedelta
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np

In [2]:
seed = 23
torch.manual_seed(seed)
np.random.seed(seed)

In [3]:
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
  
if torch.version.cuda != None:
    # Storing ID of current CUDA device
    cuda_id = torch.cuda.current_device()
    print(f"ID of current CUDA device: {torch.cuda.current_device()}")
    print(f"Name of current CUDA device: {torch.cuda.get_device_name(cuda_id)}")

Is CUDA supported by this system? True
CUDA version: 11.7
ID of current CUDA device: 0
Name of current CUDA device: NVIDIA GeForce GTX 1070


## Full VAE model test

In [4]:
dir_results = os.path.join('models','full_HSI_AD')
load_weights = False
training_data_index = 11

data_path = os.path.join("data")
df_files = create_df_from_files_in_path(data_path, verbose = False)
anomaly_map, hs_image = load_HSI_from_idx(training_data_index, df_files, verbose = True)
height, width, layers = np.shape(hs_image)

VAE = VAE_model_trainer_full_HSI.HSI_convolutional_anomaly_detector(
                                         dir_results, 
                                         zdim=8192, 
                                         load_weights=load_weights, 
                                         input_shape=(layers, height, width), 
                                        #  loss_reconstruction='bce_logits', 
                                         loss_reconstruction='weighted_l2',
                                        #  loss_reconstruction='bce',
                                        #  loss_reconstruction='l2',
                                         loss_kl='mean',
                                         level_cams=0, 
                                         verbose = 0
                                         )

print(VAE.conv_encoder)
print(VAE.conv_decoder)



Reading HSI image from data\abu-urban-4.mat...
Hypercube abu-urban-4 data structure generated with dimensions 100x100x205.
Anomaly map data structure generated with dimensions 100x100.
Encoder(
  (conv0): Conv2d(205, 128, kernel_size=(1, 1), stride=(1, 1))
  (norm0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(128, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (norm1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (norm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (norm4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine

### Train

In [8]:
if load_weights == False:
    VAE.train(training_data_index, 
            data_path, 
            epochs = 5000, 
            lr=5*1e-5, 
            alpha_entropy=0,#0.1, 
            alpha_spectral_angle=1,#10,
            alpha_kl=100,#0.2,
            update_weights = 100,
            alpha_lr=0.01,
            pre_training_epochs=0,
            train_verbose = 0)

[INFO] Epoch 1/5000: RL=110683.6719 || SAD=1.2325 || KL=0.0030, ET= 1.3394477367401123 
[INFO] Epoch 2/5000: RL=107636.2188 || SAD=1.2101 || KL=0.0017, ET= 2.1372838020324707 
[INFO] Epoch 3/5000: RL=103077.2969 || SAD=1.1431 || KL=0.0010, ET= 2.9371438026428223 
[INFO] Epoch 4/5000: RL=100844.1406 || SAD=1.0863 || KL=0.0006, ET= 3.7440130710601807 
[INFO] Epoch 5/5000: RL=98628.1406 || SAD=1.1143 || KL=0.0005, ET= 4.555814504623413 
[INFO] Epoch 6/5000: RL=94990.4141 || SAD=1.0771 || KL=0.0004, ET= 5.355671644210815 
[INFO] Epoch 7/5000: RL=91742.0156 || SAD=1.0522 || KL=0.0003, ET= 6.155560731887817 
[INFO] Epoch 8/5000: RL=89654.8047 || SAD=1.0488 || KL=0.0003, ET= 6.9524006843566895 
[INFO] Epoch 9/5000: RL=85939.2500 || SAD=1.0491 || KL=0.0003, ET= 7.762264251708984 
[INFO] Epoch 10/5000: RL=83357.7812 || SAD=1.0754 || KL=0.0003, ET= 8.573092460632324 
[INFO] Epoch 11/5000: RL=80669.1250 || SAD=1.1023 || KL=0.0003, ET= 9.37092900276184 
[INFO] Epoch 12/5000: RL=77928.3281 || SAD=1

### Evaluate

In [9]:
anomaly_map, hs_image = load_HSI_from_idx(training_data_index, df_files, verbose = True)
hs_image = (hs_image/hs_image.max())
hs_image = torch.from_numpy(np.float32(hs_image))
hs_image = torch.permute(hs_image, (2, 0, 1))
hs_image = torch.unsqueeze(hs_image, dim=0)

if torch.cuda.is_available():
    hs_image = hs_image.to('cuda')
print(f'    Shape of input sample image: {np.shape(hs_image)}')

# VAE.conv_encoder.eval()
# VAE.conv_decoder.eval()

# Encode the image
# z, z_mu, z_logvar, act_maps = VAE.conv_encoder(hs_image)
# print(f'    Shape of latent sample vector: {np.shape(z)}')
# # Decode the latent vector
# out, act_map = VAE.conv_decoder(z)
# print(f'    Output HSI image shape: {np.shape(out)}')
# Compute anomaly detection score
print('\n')
VAE.verbose = 1
# VAE.level_cams = -1

score, m_hat, out, am_list_enc, am_list_dec = VAE.predict_score(hs_image)
print(f'     Anomaly detection score = {score}')

# Save the reconstruction
# save_path = os.path.join('models','attention_map')
# np.save(save_path, m_hat)
# save_path = os.path.join('models','encoder_maps')
# np.save(save_path, am_list_enc)
# save_path = os.path.join('models','decoder_maps')
# np.save(save_path, am_list_dec)

# out = torch.sigmoid(out)

# out_np = torch.permute(out, (0,2,3,1))
# out_np = out_np.cpu().detach().numpy()
# out_np = np.squeeze(out_np, axis=0)
# save_path = os.path.join('models','reconstructed_img')
# np.save(save_path, out_np)


## SAVING EVALUATION RESULTS
# Save the reconstruction
save_path = os.path.join('models','full_HSI_AD',f'attention_map_{df_files.Filename[training_data_index]}')
np.save(save_path, m_hat)
save_path = os.path.join('models','full_HSI_AD',f'encoder_maps_{df_files.Filename[training_data_index]}')
np.save(save_path, am_list_enc)
save_path = os.path.join('models','full_HSI_AD',f'decoder_maps_{df_files.Filename[training_data_index]}')
np.save(save_path, am_list_dec)
out = torch.sigmoid(out)
out_np = torch.permute(out, (0,2,3,1))
out_np = out_np.cpu().detach().numpy()
out_np = np.squeeze(out_np, axis=0)
save_path = os.path.join('models','full_HSI_AD',f'reconstructed_img_{df_files.Filename[training_data_index]}')
np.save(save_path, out_np)

Reading HSI image from data\abu-urban-4.mat...
Hypercube abu-urban-4 data structure generated with dimensions 100x100x205.
Anomaly map data structure generated with dimensions 100x100.
    Shape of input sample image: torch.Size([1, 205, 100, 100])


Attention mask extracted, with shape torch.Size([1, 100, 100]).
Attention mask restored, with shape (100, 100).
Score calculated, with value 0.0018638945184648037.
     Anomaly detection score = 0.0018638945184648037


c:\Users\telmo\Desktop\Documentación VIU\09 - TFM Imágenes Hiperespectrales\Anomaly Detection\anomaly_detection_code\VAE_model_trainer_full_HSI.py:284: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  z, z_mu, z_logvar, f = self.conv_encoder(torch.tensor(x).cuda().float())


In [10]:
if load_weights == False:
    reconstruction_losses = torch.stack(VAE.lr_lc).cpu().detach().numpy()
    KL_div_losses = torch.stack(VAE.lkl_lc).cpu().detach().numpy()
    # entropy_losses = torch.stack(VAE.H_lc).cpu().detach().numpy()
    spectral_angle_losses = torch.stack(VAE.lsa_lc).cpu().detach().numpy()
    history = pd.DataFrame(list(zip(reconstruction_losses, KL_div_losses, spectral_angle_losses)),
                                columns=['reconstruction_losses', 'KL_div_losses', 'spectral_angle_losses'])
    save_path = os.path.join('models','full_HSI_AD',f'training_losses_{df_files.Filename[training_data_index]}.csv')
    history.to_csv(save_path, index=False)

In [ ]:
anomaly_map, hs_image = load_HSI_from_idx(training_data_index, df_files, verbose = True)
hs_image = (hs_image/hs_image.max())
hs_image = torch.from_numpy(np.float32(hs_image))
# random_img = torch.from_numpy(np.float32(generate_random_HSI(5, 5, 207)))

hs_image = torch.permute(hs_image, (2, 0, 1))
hs_image = torch.unsqueeze(hs_image, dim=0)

if torch.cuda.is_available():
    hs_image = hs_image.to('cuda')
print(f'    Shape of input sample image: {np.shape(hs_image)}')

score, m_hat, img_hat, am_list_enc, am_list_dec = VAE.predict_score(hs_image)